In [ ]:
# Look at how Francis joined the two datasets and see if that works...

# TODO: 12/2

1. Fix Francis' matplotlib
2. Set up private github repository
6. Add in Jacob's fix to Head to Head (it's like 2 lines)
8. Make ROC/AUC plots
9. Bring in Jacob's point-by-point model.
10. Website
11. Explanations in ipython Notebook

# Data Clean-up

In [176]:
#!pip install git+https://github.com/sublee/elo
#!pip install git+https://github.com/sublee/glicko2

In [2]:
# Imports necessary libraries
import pandas as pd
import numpy as np
import datetime
from sklearn import linear_model
import math
from operator import itemgetter, attrgetter
import elo
import glicko2
import trueskill
import matplotlib
import re
matplotlib.use('TkAgg')

import matplotlib as mpl
import matplotlib.cm as cm

import matplotlib.pyplot as plt
import seaborn as sns
from __future__ import division

In [12]:
''' Combines CSV 1999-2015 complete match files and constructs 1 dataframe.
    Choose 1999 as starting year because that's when Federer starts.
    We can change this if we want more data.
'''
atp_year_list = []
for i in xrange(1999,2016):
    atp_year_list.append(pd.read_csv("tennis_data/atp_matches_{0}.csv".format(i)))
atp_all_matches = pd.concat(atp_year_list, ignore_index = True)

# Set up the pbp (point by point) column that will contain the pbp information for matches we have info on
atp_all_matches['pbp'] = [None]*atp_all_matches.shape[0]
atp_all_matches = atp_all_matches.rename(columns={'winner_name':'w_name','loser_name':'l_name'})
atp_all_matches['w_name'] = [x.replace('-',' ') for x in atp_all_matches['w_name']]
atp_all_matches['l_name'] = [x.replace('-',' ') for x in atp_all_matches['l_name']]
atp_all_matches['tourney_name'] = [x.partition('Davis Cup')[0] for x in atp_all_matches['tourney_name']]
atp_all_matches['tourney_name'] = [s if s!='' else 'Davis Cup' for s in atp_all_matches['tourney_name']]
atp_all_matches.shape

(53054, 50)

In [13]:
set(atp_all_matches['tourney_name'])

{"'s-Hertogenbosch",
 'ATP Challenger Tour Finals CH',
 'Acapulco',
 'Adelaide',
 'Amersfoort',
 'Amsterdam',
 'Athens Olympics',
 'Atlanta',
 'Auckland',
 'Australian Open',
 'Bangkok',
 'Barcelona',
 'Basel',
 'Bastad',
 'Beijing',
 'Beijing Olympics',
 'Belgrade',
 'Bogota',
 'Boston',
 'Bournemouth',
 'Brighton',
 'Brisbane',
 'Bucharest',
 'Buenos Aires',
 'Canada Masters',
 'Casablanca',
 'Chennai',
 'Cincinnati',
 'Cincinnati Masters',
 'Copenhagen',
 'Costa Do Sauipe',
 'Curitiba CH',
 'Davis Cup',
 'Delray Beach',
 'Doha',
 'Dubai',
 'Dusseldorf',
 'Eastbourne',
 'Estoril',
 'Geneva',
 'Grand Slam Cup',
 'Gstaad',
 'Halle',
 'Hamburg',
 'Hamburg Masters',
 'Ho Chi Minh City',
 'Hong Kong',
 'Houston',
 'Indian Wells',
 'Indian Wells Masters',
 'Indianapolis',
 'Istanbul',
 'Johannesburg',
 'Kitzbuhel',
 'Kuala Lumpur',
 'Las Vegas',
 'London',
 'London Olympics',
 'Long Island',
 'Los Angeles',
 'Lyon',
 'Madrid',
 'Madrid Masters',
 'Mallorca',
 'Marseille',
 'Masters Cup',
 

In [122]:
atp_all_matches.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,pbp
0,1999-773,Bucharest,Clay,32,A,19990927,1,102374,1.0,NaN,...,2.0,1.0,111.0,64.0,42.0,24.0,14.0,10.0,12.0,None
1,1999-773,Bucharest,Clay,32,A,19990927,2,102533,NaN,NaN,...,1.0,3.0,80.0,49.0,26.0,17.0,10.0,7.0,10.0,None
2,1999-773,Bucharest,Clay,32,A,19990927,3,103507,NaN,NaN,...,1.0,2.0,71.0,53.0,20.0,10.0,9.0,10.0,17.0,None
3,1999-773,Bucharest,Clay,32,A,19990927,4,102223,NaN,NaN,...,0.0,3.0,48.0,28.0,16.0,6.0,8.0,4.0,9.0,None
4,1999-773,Bucharest,Clay,32,A,19990927,5,103292,NaN,NaN,...,3.0,6.0,93.0,44.0,30.0,17.0,13.0,11.0,17.0,None


In [123]:
''' More dataframe preparation for combining dateframes.
    Matches detailed match info for select matches to correct row in complete match list
    If date is within 30 days of each other is correct, players are correct,
    result is same, and first set score is same, then it's the same match. '''

# Get score into the same string format (a couple scores are mis-recorded),
# so that's why I don't match on the entire score string
atp_all_matches['score'] = atp_all_matches['score'].astype(str)

# Get dates into the same format
atp_all_matches['tourney_date'].apply(lambda x: (str(x)[8:16]))
atp_all_matches['tourney_date'] = atp_all_matches['tourney_date'].apply(lambda x: datetime.datetime.strptime(str(x), "%Y%m%d"))
atp_all_matches['tourney_date'] = [x.date() for x in atp_all_matches['tourney_date']]
#atp_all_matches['match_year'] = atp_all_matches['tourney_date1'].apply(lambda x: x.year)
#atp_all_matches['match_month'] = atp_all_matches['tourney_date1'].apply(lambda x: x.month)
atp_all_matches['score_start'] = atp_all_matches['score'].apply(lambda x: x[:3])
atp_all_matches = atp_all_matches.sort(['tourney_date'], ascending=1).reset_index()
del atp_all_matches['index']
atp_all_matches['match_year'] = [x.year for x in atp_all_matches['tourney_date']]
atp_all_matches['match_month'] = [x.month for x in atp_all_matches['tourney_date']]
atp_all_matches['score'] = [re.sub(r"[\(\[].*?[\)\]]", "", s) for s in atp_all_matches['score']]
atp_all_matches.head()

/Users/jacobgollub/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:17: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,pbp,score_start,match_year,match_month
0,1999-451,Doha,Hard,32,A,1999-01-04,20,102223,NaN,WC,...,52.0,38.0,19.0,16.0,2.0,7.0,None,6-2,1999,1
1,1999-451,Doha,Hard,32,A,1999-01-04,3,102950,NaN,NaN,...,27.0,16.0,7.0,7.0,2.0,6.0,None,6-1,1999,1
2,1999-451,Doha,Hard,32,A,1999-01-04,4,101543,NaN,NaN,...,32.0,22.0,10.0,9.0,4.0,7.0,None,6-3,1999,1
3,1999-451,Doha,Hard,32,A,1999-01-04,5,102338,3.0,NaN,...,60.0,38.0,17.0,13.0,11.0,16.0,None,2-6,1999,1
4,1999-451,Doha,Hard,32,A,1999-01-04,6,102271,NaN,NaN,...,46.0,31.0,30.0,14.0,11.0,16.0,None,1-6,1999,1


In [124]:
''' Combine all the matches that have pbp (point by point) information into one dataframe
    and clean up columns in preparation for matching with the all_atp_matches dataframe.
'''

pbp_matches_archive = pd.read_csv("pbp_matches_atp_main_archive.csv")
pbp_matches_current = pd.read_csv("pbp_matches_atp_main_current.csv")
pbp_matches = pd.concat([pbp_matches_archive,pbp_matches_current])
pbp_matches.winner = pbp_matches.winner - 1
pbp_matches = pbp_matches.reset_index()
del pbp_matches['index']
#pbp_matches['date'] = [str(x).split(' ')[0].replace('-','') for x in pbp_matches['date']]
pbp_matches['w_name'] = np.where(pbp_matches['winner'] == 0, pbp_matches['server1'], pbp_matches['server2'])
pbp_matches['l_name'] = np.where(pbp_matches['winner'] == 0, pbp_matches['server2'], pbp_matches['server1'])
pbp_matches['w_name'] = [x.replace('-',' ') for x in pbp_matches['w_name']]
pbp_matches['l_name'] = [x.replace('-',' ') for x in pbp_matches['l_name']]
pbp_matches['date'] = pd.to_datetime(pbp_matches['date'])
pbp_matches['match_year'] = pbp_matches['date'].apply(lambda x: x.year)
pbp_matches['match_month'] = pbp_matches['date'].apply(lambda x: x.month)
pbp_matches['score_start'] = pbp_matches['score'].apply(lambda x: x[:3])
pbp_matches['date'] = [x.date() for x in pbp_matches['date']]
pbp_matches['score'] = [re.sub(r"[\(\[].*?[\)\]]", "", s) for s in pbp_matches['score']]


# Makes pbp_matches dataframe only contain the matching columns and the new column we want "pbp"
#pbp_matches = pbp_matches.iloc[:,[7,10,11,12,13,14]]

''' Here, we actually add the 'pbp' column to the correct row in the other dataframe,
    based on criteria I chose that should be the same match,
    ie. correct winner and loser, same first set score, and match dates
    are within 30 days of one another.
'''

" Here, we actually add the 'pbp' column to the correct row in the other dataframe,\n    based on criteria I chose that should be the same match,\n    ie. correct winner and loser, same first set score, and match dates\n    are within 30 days of one another.\n"

In [125]:
print pbp_matches.shape
print len(set(pbp_matches['pbp'].values))

(11663, 15)
11315


## Rating Systems

### Elo, Trueskill, Glicko
1. http://gobase.org/studying/articles/elo/
2. http://www.gamefaqs.com/boards/610657-dota-2/67994646
3. https://github.com/sublee/elo/blob/master/elo.py
4. http://stephenwan.net/thoughts/2012/10/02/elo-rating-system.html
5. https://deltadata.wordpress.com/2014/01/11/glicko-2-for-tennis-part-2-the-model/comment-page-1/#comment-192
6. https://github.com/sublee/glicko2/blob/master/glicko2.py
7. http://trueskill.org/
8. https://pypi.python.org/pypi/trueskill


## Add the correct Elo, Trueskill, and Glicko ratings at each match

In [126]:
atp_all_matches.shape

(53054, 53)

In [127]:
''' Make a dict of players with elo ratings'''
elo_prior = elo.Rating()
players_list = np.union1d(atp_all_matches.w_name.values, atp_all_matches.l_name.values)
players_elo = dict(zip(list(set(players_list)), [elo_prior] *len(players_list)))

mean_elo, diff_elo = [], []
elo_1s, elo_2s = [],[]

for i, row in atp_all_matches.iterrows():
    # Find Rating in dictionary
    elo1 = players_elo[str(row['w_name'])]
    elo2 = players_elo[str(row['l_name'])]
    
    # Append Rating to List for adding to DataFrame
    mean_elo.append((elo1.value + elo2.value)/2)
    diff_elo.append(float(elo1.value) - float(elo2.value))
    elo_1s.append(elo1.value)
    elo_2s.append(elo2.value)


    new_elo1, new_elo2 = elo.rate_1vs1(elo1,elo2)

    # Update the new Elo ratings in dictionary
    players_elo[str(row['w_name'])] = elo.Rating(new_elo1)
    players_elo[str(row['l_name'])] = elo.Rating(new_elo2)

#Add columns
#atp_all_matches['mean_elo'] = pd.Series(mean_elo, index = atp_all_matches.index)
#atp_all_matches['diff_elo'] = pd.Series(diff_elo, index = atp_all_matches.index)
atp_all_matches['w_elo'] = pd.Series(elo_1s, index = atp_all_matches.index)
atp_all_matches['l_elo'] = pd.Series(elo_2s, index = atp_all_matches.index)

In [128]:
columns = ['tourney_id','tourney_name','tourney_date' ,'match_num','w_name','l_name',
                'winner_id','loser_id','best_of','w_elo','l_elo','match_year','match_month','score']

In [129]:
atp_all_matches = atp_all_matches[columns]

In [143]:
# Now, just make a dictionary with keys 'w_name'+'l_name'+'match_year'+'score' to connect pbp 
# strings to atp_all_matches
# replacing dashes with spaces accounted for over 1000 more pbp matches
# other discrepancies due to tiebreak scores, or matches in pbp being qualies
# MUST REMOVE PARENTHESES TERMS FROM TIE-BREAKS IN SCORES

pbp_dict = {}; winner_dict = {}
for i in xrange(len(pbp_matches)):
    key = pbp_matches['w_name'][i] +' ' +  pbp_matches['l_name'][i] + ' ' \
        + str(pbp_matches['match_year'][i]) + ' ' + pbp_matches['score'][i]
    pbp_dict[key] = pbp_matches['pbp'][i]
    winner_dict[key] = pbp_matches['winner'][i]

In [144]:
c = 0
pbps = []
winners = []
info = {}

for i in xrange(len(atp_all_matches)):
    key = atp_all_matches['w_name'][i] +' ' +  atp_all_matches['l_name'][i] + ' ' \
        + str(atp_all_matches['match_year'][i]) + ' ' + atp_all_matches['score'][i]
    if key in pbp_dict:
        c += 1
        pbps.append(pbp_dict[key])
        winners.append(winner_dict[key])
        info[key] = 1
    else:
        pbps.append('NA')
        winners.append('NA')
print c
atp_all_matches['pbp'] = pbps
atp_all_matches['winner'] = winners

9563


In [ ]:
atp_all_matches

In [87]:
'''
for i in xrange(len(pbp_matches)):
    key = pbp_matches['w_name'][i] +' ' +  pbp_matches['l_name'][i] + ' ' \
        + str(pbp_matches['match_year'][i]) + ' ' + pbp_matches['score'][i]
    if key not in info:
        print key
'''

Evgeny Donskoy Andrey Kuznetsov 2010 7-5 7-6(1)
Robin Soderling Gael Monfils 2010 6-1 7-6(0)
Richard Gasquet Bjorn Phau 2011 6-1 6-7(5) 7-6(3)
Janko Tipsarevic Alexandre Kudryavtsev 2011 6-2 7-6(2)
Stanislas Wawrinka David Goffin 2011 7-6(3) 7-6(2)
Peter Luczak Alexander Peya 2011 4-6 6-1 6-4
Feliciano Lopez Philipp Petzschner 2011 6-4 7-6(10)
Peter Luczak Dimitry Tursinov 2011 4-6 7-6(6) 6-4
Denis Istomin Thiemo de Bakker 2011 7-6(5) 6-4
Richard Berankis Peter Luczak 2011 6-3 5-7 6-2
Matthew Ebden Greg Jones 2011 6-4 6-1
Robin Soderling Michael Berrer 2011 6-3 7-6(6)
Pere Riba Jose Statham 2011 4-6 6-4 6-3
Philipp Petzschner Thiemo de Bakker 2011 6-1 6-7(5) 7-6(2)
Pablo Cuevas Robin Haase 2011 6-3 6-7(3) 7-6(2)
Juan Martin del Potro Feliciano Lopez 2011 6-7(5) 7-6(9) 7-6(2)
Florian Mayer Juan Martin del Potro 2011 6-2 7-5
Lleyton Hewitt Ruben Bemelmans 2011 6-4 6-3
Novak Djokovic Andrey Golubev 2011 4-6 6-3 6-1
John Isner Nicolas Mahut 2011 6-3 7-6(4)
Ruben Bemelmans Andrey Golubev 20

In [147]:
a = atp_all_matches[atp_all_matches['w_name']=='Jack Sock']
#a = a[a['l_name']=='Jurgen Zopp']
#a.sort('match_num')

In [172]:
df = atp_all_matches[atp_all_matches['pbp']!='NA']
cols = df.columns.drop(['loser_id','winner_id'])
df = df[cols]
df.to_csv('elo_pbp.csv')